In [1]:
import os

import pandas
import networkx

import do_tools

In [2]:
# read the disease ontology
path = os.path.join('download', 'HumanDO.obo')
do = do_tools.load_do(path)
dox = do_tools.do_to_networkx(do)


In [3]:
# read the slim disease ontology terms
path = os.path.join('data', 'slim-terms.tsv')
slim_df = pandas.read_table(path)
slim_df[:3]

,doid,name,source,pathophysiology
0,DOID:2531,Hematologic cancer,DOcancerslim,neoplastic
1,DOID:1319,Brain cancer,DOcancerslim,neoplastic
2,DOID:1324,Lung cancer,DOcancerslim,neoplastic


In [4]:
# Convert DO slim names to those in the ontology
id_to_name = {node.id: node.name for node in dox}
slim_df.name = slim_df.doid.map(id_to_name).tolist()
slim_df.to_csv(path, sep='\t', index=False)

In [ ]:
# check for terms that in slim but not in the DO
all_doids = set(do.get_term_ids())
slim_doids = set(slim_df.doid)
unmatched = slim_doids - all_doids
slim_doids &= all_doids
slim_df.loc[slim_df.doid.isin(unmatched)]

In [ ]:
# check for node redundancy
slim_terms = set(map(do.get_term, slim_doids))
for term in slim_terms:
    nodes_to_root = networkx.descendants(dox, term)
    conflicts = {x.name for x in nodes_to_root & slim_terms}
    if conflicts:
        print(term.name, conflicts)

In [ ]:
path = os.path.join('data', 'xrefs.tsv')
map_unprop_df = pandas.read_table(path)

path = os.path.join('data', 'xrefs-prop.tsv')
map_prop_df = pandas.read_table(path)

In [ ]:
slim_df = slim_df.rename(columns={'doid': 'doid_code'})
slim_map_unprop_df = slim_df[['doid_code']].merge(map_unprop_df)
slim_map_prop_df = slim_df[['doid_code']].merge(map_prop_df)
slim_map_prop_df[:3]

In [ ]:
path = os.path.join('data', 'xrefs-slim.tsv')
slim_map_unprop_df.to_csv(path, sep='\t', index=False)

path = os.path.join('data', 'xrefs-prop-slim.tsv')
slim_map_prop_df.to_csv(path, sep='\t', index=False)

In [ ]:
rows = list()
for term in slim_terms:
    subsumed = networkx.ancestors(dox, term)
    row_part = [term.id, term.name]
    rows.append(row_part + row_part + [0])
    for subterm in subsumed:
        distance = networkx.shortest_path_length(dox, subterm, term)
        rows.append(row_part + [subterm.id, subterm.name, distance])
rows.sort()

slim_prop_df = pandas.DataFrame(rows, columns=['slim_id', 'slim_name', 'subsumed_id', 'subsumed_name', 'min_distance'])
path = os.path.join('data', 'slim-terms-prop.tsv')
slim_prop_df.to_csv(path, sep='\t', index=False)